In [20]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
#from utils import label_accuracy_score
import cv2

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
#from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.8.1+cu101
GPU 사용 가능 여부: True
Tesla T4
1


## baseline model

### <font color='red'>[TODO] 코드 구현 FCN-32s </font>

In [21]:
# 모델 참고 코드 
# https://github.com/wkentaro/pytorch-fcn/
import torch
import torch.nn as nn
class FCN8s(nn.Module):
    def __init__(self, num_classes=12):
        super(FCN8s, self).__init__()
        self.relu    = nn.ReLU(inplace=True)

        self.conv1_1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        self.conv1_2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        
        self.conv2_1 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        self.conv2_2 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        
        self.conv3_1 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        self.conv3_2 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        self.conv3_3 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        
        self.conv4_1 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        self.conv4_2 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        self.conv4_3 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        
        self.conv5_1 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        self.conv5_2 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')
        self.conv5_3 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride=1, padding=1, bias=True, padding_mode='zeros')

        self.conv6_1 = nn.Conv2d(in_channels = 512, out_channels = 4096, kernel_size = 1, stride=1, padding=0, bias=True, padding_mode='zeros')

        self.conv7_1 = nn.Conv2d(in_channels = 4096, out_channels = 4096, kernel_size = 1, stride=1, padding=0, bias=True, padding_mode='zeros')

        self.score = nn.Conv2d(in_channels=4096, out_channels = num_classes, kernel_size = 1, stride=1, padding=0, bias=True)

        self.maxPool = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.upscore32 = nn.ConvTranspose2d(in_channels=num_classes, out_channels = num_classes, kernel_size = 64, stride=32, padding=16)
        '''
        [TODO]

        ''' 

    def forward(self, x):

        x = self.conv1_1(x)
        x = self.relu(x)
        x = self.conv1_2(x)
        x = self.relu(x)
        x = self.maxPool(x)

        
        x = self.conv2_1(x)
        x = self.relu(x)
        x = self.conv2_2(x)
        x = self.relu(x)
        x = self.maxPool(x)

        
        x = self.conv3_1(x)
        x = self.relu(x)
        x = self.conv3_2(x)
        x = self.relu(x)
        x = self.conv3_3(x)
        x = self.relu(x)
        x = self.maxPool(x)

        
        x = self.conv4_1(x)
        x = self.relu(x)
        x = self.conv4_2(x)
        x = self.relu(x)
        x = self.conv4_3(x)
        x = self.relu(x)
        x = self.maxPool(x)

        
        x = self.conv5_1(x)
        x = self.relu(x)
        x = self.conv5_2(x)
        x = self.relu(x)
        x = self.conv5_3(x)
        x = self.relu(x)
        x = self.maxPool(x)


        x = self.conv6_1(x)
        x = self.relu(x)

        x = self.conv7_1(x)
        x = self.relu(x)
        
        x = self.score(x)

        output = self.upscore32(x)
        
        return output

In [22]:
# 구현된 model에 임의의 input을 넣어 output이 잘 나오는지 test

model = FCN8s(num_classes=12)
x = torch.randn([1, 3, 512, 512])
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())

model = model.to(device)

input shape :  torch.Size([1, 3, 512, 512])
output shape :  torch.Size([1, 12, 512, 512])
